In [1]:
import torch
torch.__version__

'2.0.1+cu118'

In [ ]:
!pip install torch==1.13.0+cu116 torchvision==0.14.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 612.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2+cu118
    Uninstalling torchvision-0.15.2+cu118:
      Successfully uninstalled torchvision-0.15.2+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.0+cu116 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.0+cu116 which is incompatible.
torchtext 0.15.2 requires 

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html


Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.13/index.html


In [ ]:
!git clone --branch 2.x https://github.com/open-mmlab/mmdetection.git
!cd mmdetection; python setup.py install

In [ ]:
!pip uninstall yapf -y

Found existing installation: yapf 0.40.2
Uninstalling yapf-0.40.2:
  Successfully uninstalled yapf-0.40.2


In [ ]:
!pip install yapf==0.40.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 2.8 MB/s eta 0:00:00


In [2]:
# 런타임 다시 시작
from mmdet.apis import init_detector, inference_detector
import mmcv

ModuleNotFoundError: ignored

In [3]:
# 데이터가 저장된 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from glob import glob

# birds의 일부 데이터(100개)만 이용
path = "/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft"
img_path = "/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/training/images"

train_img_list = glob(f"{img_path}/*") # 이미지 파일들의 경로 주소들을 읽어온 후 리스트로 저장

In [ ]:
from sklearn.model_selection import train_test_split

# random state 0으로 변경 <- 0번째 data부터 섞음
train_img_list, val_img_list = train_test_split(train_img_list, test_size=0.1, random_state=0)
train_img_list, test_img_list = train_test_split(train_img_list, test_size=0.1, random_state=0)

with open(f'{path}/train.txt', 'w') as f: # train을 위한 self.ann_file 만들기
    f.write('\n'.join(train_img_list) + '\n')

with open(f'{path}/val.txt', 'w') as f: # val을 위한 self.ann_file 만들기
    f.write('\n'.join(val_img_list) + '\n')

with open(f'{path}/test.txt', 'w') as f: # test을 위한 self.ann_file 만들기
    f.write('\n'.join(test_img_list) + '\n')

In [ ]:
print("train 수: {}\nvalidation 수: {}\ntest 수: {}".format(len(train_img_list), len(val_img_list), len(test_img_list)))

train 수: 441
validation 수: 55
test 수: 50


In [ ]:
import json

def getInformationsFromJson(filePath):
    with open(filePath) as file:
        bbox_names = []
        bboxes = []

        oneJsonFile = json.load(file)

        pure_image_file_name = oneJsonFile["images"][0]["id"]
        width = oneJsonFile["images"][0]["width"]
        height = oneJsonFile["images"][0]["height"]

        for annotation in oneJsonFile["annotations"]:
            category_id = int(annotation["category_id"])
            xmin = int(annotation["bbox"][0])
            ymin = int(annotation["bbox"][1])
            xmax = int(annotation["bbox"][2])
            ymax = int(annotation["bbox"][3])

            bboxes.append([xmin, ymin, xmax, ymax])
            bbox_names.append(category_id)

        return pure_image_file_name, bbox_names, bboxes

In [ ]:
label_name = "/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/training/labels/1001473_597.json"
pure_image_file_name, bbox_names, bboxes = getInformationsFromJson(label_name)
print(pure_image_file_name, bbox_names, bboxes, sep='\n')

1001473_597
[223]
[[1551, 487, 76, 28]]


In [ ]:
import copy
import os.path as osp
import mmcv
import numpy as np
import cv2

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

@DATASETS.register_module(force=True)
class AircraftDataset(CustomDataset):
    CLASSES = ("Pigeon", "Seagull", "Magpie", "Crow", "Parrot", "Goose", "Duck", "Mallard", "Heron", "Sparrow", "Swan",
               "Airplane", "Helicopter", "FighterPlane", "Paragliding", "Drone")

    # annotation에 대한 모든 파일명을 가지고 있는 텍스트 파일을 __init__(self, ann_file)로 입력 받고,
    # 이 self.ann_file이 load_annotations()의 인자로 입력

    def load_annotations(self, ann_file):
        cat2label = {1: 'Pigeon', 2: 'Seagull', 3: 'Magpie', 4: 'Crow', 5: 'Parrot', 6: 'Goose', 7: 'Duck', 8: 'Mallard', 9: 'Heron', 10: 'Sparrow', 11: 'Swan',
                     223:"Airplane", 224:"Helicopter", 225:"FighterPlane", 226:"Paragliding", 227:"Drone"
                     }
        image_file_address_path_list = mmcv.list_from_file(self.ann_file)
        # 포맷 중립 데이터를 담을 list 객체
        data_infos = []

        for image_file_address_path in image_file_address_path_list:
            # image를 직접 로드하여 원본 이미지의 너비, 높이 값 구함
            image = cv2.imread(image_file_address_path)
            height, width = image.shape[:2]

            # 개별 annotation json 파일이 있는 서브 디렉토리의 prefix 변환
            annotation_json_file_address_path = image_file_address_path.replace('images', 'labels')
            annotation_json_file_address_path = annotation_json_file_address_path.replace('jpg', 'json')
            label_prefix = self.img_prefix.replace('images', 'labels')

            # getInformationsFromJson()를 이용하여 개별 json 파일에 있는 이미지의 모든 bbox 정보를 list 객체로 생성
            pure_image_file_name, bbox_names, bboxes = getInformationsFromJson(annotation_json_file_address_path)

            # 개별 annotation json 파일을 1개 line 씩 읽어서 list 로드.
            anno_json_file = osp.join(label_prefix, str(pure_image_file_name) + 'json')
            # 이미지 파일은 있으나 json annotation 파일이 없는 경우는 제외.
            if not osp.exists(anno_json_file):
                continue

            # 개별 image의 annotation 정보 저장용 Dict 생성. key값 filename에는 image의 파일명만 들어감(디렉토리는 제외)
            # 영상에는 data_info = {'filename': filename} 으로 되어 있으나 filename은 image 파일명만 들어가는게 맞음.
            data_info = {'filname': str(pure_image_file_name) + '.jpg',
                         'width': width, 'height': height}

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            # bbox별 Object들의 clsas name을 class id로 매핑. class id는 tuple(list)형의 CLASSES의 index값에 따라 설정
            for bbox_name, bbox in zip(bbox_names, bboxes):
                # 만약 bbox_name이 클래스명에 해당되면, gt_bboxes와 gt_labels에 추가, 그렇지 않으면 gt_bboxes_ignore, gt_labels_ignore에 추가
                # bbox_name이 CLASSES 중에 반드시 하나 있어야 함. 안 그러면 FILTERING 되므로 주의 할 것.
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    # gt_labels에는 class id를 입력
                    if bbox_name in range(1, 12): # 조류에 해당
                        gt_labels.append(bbox_name-1)
                    if bbox_name in range(223, 228): # 비행객체에 해당
                        gt_labels.append(bbox_name-223+11)
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels.ignore.append(-1)

                # 개별 image별 annotation 정보를 가지는 Dict 생성. 해당 Dict 생성. 해당 Dict의 value값을 np.array형태로 bbox의 좌표와 label값으로 생성.
                data_anno = {
                    'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    'labels': np.array(gt_labels, dtype=np.long),
                    'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
                    'labels_ignore': np.array(gt_labels_ignore, dtype=np.long)
                }

                # image에 대한 메타 정보를 가지는 data_info Dict에 'ann' key값으로 data_anno를 value로 저장.
                data_info.update(ann=data_anno)
                # 전체 annotation 파일들에 대한 정보를 가지는 data_infos 에 data_info Dict를 추가
                data_infos.append(data_info)
                #print (data_info)

            return data_infos

In [ ]:
# config 파일은 ssd500 backbone 사용.
# 학습으로 생성된 모델을 Google Drive에 저장
#config_file = "/content/mmdetection/configs/ssd/ssd300_fp16_coco.py"
#checkpoint_file = "/content/mmdetection/checkpoints/ssd300_coco_20210803_015428-d231a06e.pth"
#config_file = "/content/mmdetection/configs/ssd/ssd512_fp16_coco.py"
#checkpoint_file = "/content/mmdetection/checkpoints/ssd512_coco_20210803_022849-0a47a1ca.pth"

config_file = '/content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = '/content/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

In [ ]:
# !cd mmdetection; mkdir checkpoints
# # ssd 폴더의 README.md 클릭 -> Download의 model 클릭 -> url 복사
# !wget -O ./mmdetection/checkpoints/ssd512_coco_20210803_022849-0a47a1ca.pth  ssd300_coco_20210803_015428-d231a06e.pth

--2023-10-12 18:13:16--  http://ssd300_coco_20210803_015428-d231a06e.pth/
Resolving ssd300_coco_20210803_015428-d231a06e.pth (ssd300_coco_20210803_015428-d231a06e.pth)... failed: Name or service not known.
wget: unable to resolve host address ‘ssd300_coco_20210803_015428-d231a06e.pth’


In [ ]:
# !cd mmdetection; mkdir checkpoints
# !wget -O ./mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

mkdir: cannot create directory ‘checkpoints’: File exists
--2023-10-12 18:51:14--  http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 8.48.85.213, 163.181.66.110, 163.181.66.106, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|8.48.85.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167287506 (160M) [application/octet-stream]
Saving to: ‘./mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’

./mmdetection/check 100%[===================>] 159.54M  34.5MB/s    in 5.0s    

2023-10-12 18:51:19 (32.2 MB/s) - ‘./mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’ saved [167287506/167287506]



In [ ]:
#soft link로 Google Drive Directory 연결.
!ln -s /content/gdrive/My\ Drive/ /mydrive

ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [ ]:
# Google Drive 밑에 Directory 생성. 이미 생성 되어 있을 시 오류 발생.
!mkdir "/mydrive/aircraft_work_dir"

mkdir: cannot create directory ‘/mydrive/aircraft_work_dir’: File exists


In [ ]:
!nvidia-smi # 할당된 GPU 확인

Thu Oct 12 19:36:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg)
print(cfg.pretty_text)

Config (path: /content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py): {'model': {'type': 'FasterRCNN', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}}, 'roi_head': {'type': 'StandardRoIHead', 'bbox_ro

In [ ]:
!ls -ali "/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/"

total 53
5123 -rw------- 1 root root  4488 Oct 12 19:35 test.txt
  43 drwx------ 4 root root  4096 Oct 12 11:14 training
5117 -rw------- 1 root root 39569 Oct 12 19:35 train.txt
5120 -rw------- 1 root root  4935 Oct 12 19:35 val.txt


In [ ]:
!pwd

/content


In [ ]:
from mmdet.apis import set_random_seed

# dataset에 대한 환경 파라미터 수정
cfg.dataset_type = 'AircraftDataset'
cfg.data_root = '/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/'

# train, val, test dataset에 대한 type, data_root, ann_file, img_prefix 환경 파라미터 수정.
cfg.data.train.dataset.type = 'AircraftDataset'
cfg.data.train.dataset.data_root = '/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/'
cfg.data.train.dataset.ann_file = 'train.txt'
cfg.data.train.dataset.img_prefix = 'training/images'

cfg.data.val.type = 'AircraftDataset'
cfg.data.val.data_root = '/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/'
cfg.data.val.ann_file = 'val.txt'
cfg.data.val.img_prefix = 'training/images'

cfg.data.test.type = 'AircraftDataset'
cfg.data.test.data_root = '/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/'
cfg.data.test.ann_file = 'test.txt'
cfg.data.test.img_prefix = 'training/images'

# ************class의 갯수 수정 필요***********
cfg.model.bbox_head.num_classes = 16
# pretrained 모델
cfg.load_from = 'checkpoints/ssd512_coco_20210803_022849-0a47a1ca.pth'

# 학습 weigth 파일로 로그를 저장하기 위한 디렉토리로 구글 Drive 설정.
cfg.work_dif = '/mydrive/aircraft_work_dir'

# 학습율 변경 환경 파라미터 설정.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
# 5번 마다 한번씩 로그를 작성
cfg.log_config.interval = 5

cfg.runner.max_epochs = 5

# 평가 metric 설정.
cfg.evaluation.metric = 'mAP'
# 평가 metric 수행할 epoch interval 설정.
cfg.evaluation.interval = 5
# 학습 iteration시마다 모델을 저장할 epoch interval 설정.
cfg.checkpoint_config.interval = 5

# 학습 시 Batch size 설정(단일 GPU 별 Batch size로 설정됨) (gpu가 2개이면 batch size가 8이 된다.)
cfg.data.samples_per_gpu = 4

# seed 설정(결과를 reproducible 가능하게 만듦)
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_idx = range(1)
# 두번 config를 로드하면 lr_config의 policy가 사라지는 오류로 인하여 설정.
cfg.lr_config.policy = 'step'

# ConfigDict' object has no attribute 'device 오류 발생시 반드시 설정 필요. https://github.com/open-mmlab/mmdetection/issues/7901
cfg.device = 'cuda'

# We can initialize the logger for training and have a look
# at the final config used for training
# print(f'Config:\n{cfg.pretty_text}')

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# train용 Dataset 생성.
datasets = [build_dataset(cfg.data.train.dataset)]

/usr/local/lib/python3.10/dist-packages/mmdet-2.28.2-py3.10.egg/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


TypeError: ignored

In [ ]:
from mmdet.apis import set_random_seed

from mmcv import Config

cfg = Config.fromfile(config_file)
print(cfg)
print(cfg.pretty_text)

# dataset에 대한 환경 파라미터 수정.
cfg.dataset_type = 'AircraftDataset'
cfg.data_root = '/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/'

# train, val, test dataset에 대한 type, data_root, ann_file, img_prefix 환경 파라미터 수정.
cfg.data.train.type = 'AircraftDataset'
cfg.data.train.data_root = '/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/'
cfg.data.train.ann_file = 'train.txt'
cfg.data.train.img_prefix = 'training/images'

cfg.data.val.type = 'AircraftDataset'
cfg.data.val.data_root = '/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/'
cfg.data.val.ann_file = 'val.txt'
cfg.data.val.img_prefix = 'training/images'

cfg.data.test.type = 'AircraftDataset'
cfg.data.test.data_root = '/content/gdrive/MyDrive/Colab Notebooks/datasets/aircraft/'
cfg.data.test.ann_file = 'test.txt'
cfg.data.test.img_prefix = 'training/images'

# class의 갯수 수정.
cfg.model.roi_head.bbox_head.num_classes = 5
# pretrained 모델
cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

# 학습 weight 파일로 로그를 저장하기 위한 디렉토리로 구글 Drive 설정.
cfg.work_dir = '/mydrive/aircraft_work_dir'

# 학습율 변경 환경 파라미터 설정.
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None
# 5번 마다 한번씩 로그를 작성
cfg.log_config.interval = 5

cfg.runner.max_epochs = 5

# 평가 metric 설정.
cfg.evaluation.metric = 'mAP'
# 평가 metric 수행할 epoch interval 설정.
cfg.evaluation.interval = 5
# 학습 iteration시마다 모델을 저장할 epoch interval 설정.
cfg.checkpoint_config.interval = 5

# 학습 시 Batch size 설정(단일 GPU 별 Batch size로 설정됨) (gpu가 2개이면 batch size가 8이 된다.)
cfg.data.samples_per_gpu = 4

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
# 두번 config를 로드하면 lr_config의 policy가 사라지는 오류로 인하여 설정.
cfg.lr_config.policy='step'

# ConfigDict' object has no attribute 'device 오류 발생시 반드시 설정 필요. https://github.com/open-mmlab/mmdetection/issues/7901
cfg.device='cuda'

# We can initialize the logger for training and have a look
# at the final config used for training
# print(f'Config:\n{cfg.pretty_text}')

Config (path: /content/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py): {'model': {'type': 'FasterRCNN', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch', 'init_cfg': {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'num_outs': 5}, 'rpn_head': {'type': 'RPNHead', 'in_channels': 256, 'feat_channels': 256, 'anchor_generator': {'type': 'AnchorGenerator', 'scales': [8], 'ratios': [0.5, 1.0, 2.0], 'strides': [4, 8, 16, 32, 64]}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [1.0, 1.0, 1.0, 1.0]}, 'loss_cls': {'type': 'CrossEntropyLoss', 'use_sigmoid': True, 'loss_weight': 1.0}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 1.0}}, 'roi_head': {'type': 'StandardRoIHead', 'bbox_ro

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# train용 Dataset 생성.
datasets = [build_dataset(cfg.data.train)]

/usr/local/lib/python3.10/dist-packages/mmdet-2.28.2-py3.10.egg/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


TypeError: ignored